
# 안녕하세요^^ 
# AIVLE 미니 프로젝트3차 개인 미션입니다. 
# 주어진 문제를 잘 읽고 해결하시길 바랍니다.

## 올레TV 예능 콘텐츠 중 하나인 프로그램 "슈퍼맨이 돌아왔다"의 영상을 분석하고, 
## AI 모델링을 통해 코너를 분류 해보세요.

"슈퍼맨이 돌아왔다" 는 KBS에서 방영 중인 육아 예능/관찰 예능 프로그램 입니다. <br> 유명인 아빠/엄마들이 자신의 아이들을 돌보는 육아 프로그램으로 매 주 방송에는 3~4명의 출연진들이 등장한다. <br>
이번 과제는 앞에서 실습하면서 만든 모델을 활용하여 2020년 8월 방영 영상의 코너를 분류해내는 것입니다.

아래의 조건들을 참고하여 문제를 해결하세요.
> - 주어진 데이터
>> Train 데이터(실습과 동일) : jung.mp4, park.mp4, sam.mp4, sayuri.mp4<br>
>> Test 데이터 : test.mp4 - 8월 방영분
> - 제출 파일 <br>
>> ① 코드 : [개인] 미니프로젝트3차_A000000_OOO.ipynb 
    <b>('<u>전처리 함수</u>'</b>, <b>'<u> 모델 예측 결과 출력 함수</u>'</b> 포함)<br>
>> ② 모델 : [개인] 미니프로젝트3차_A000000_OOO.h5 

[과제설명]<br>
- 전처리 함수 (my_preprocessing) <br>
매개변수는  <font color="red">test_video (.mp4) 파일</font>이며, <br>
return 값은  <font color="red">ImageDataGenerator로 전처리 완료한 test 데이터셋 (test_generator)</font> 입니다.<br> 
모듈 안에는 동영상 프레임 자르기,ImageDataGenrator, 그 외 추가 전처리 과정 모두 포함되어야 합니다. <br> 
ex) <u>def my_preprocessing(TEST_VIDEO)</u> 안에 작성합니다. <br><br>
- 모델 예측 결과 출력 함수(my_model_predict) <br>
매개변수는 <font color="red">저장한 모델경로</font>와 <font color="red">test_generator</font>로 하고 
모델 예측은 fit(test_generator)를 사용합니다.<br>
모듈의 최종 결과 예측은 출연진 별로 등장하는 구간을 출력합니다.<br>
ex) <u>def my_model_predict(test_generator, MODEL_PATH)</u> 안에 작성합니다. <br>



---

## 0. 환경 설정하기

### 1) 구글 드라이브 연결하기
아래 코드를 실행하세요.

In [ ]:
# 코랩 사용 시 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

### 2) 경로 확인하기
- "WORK_SPACE" 에 본인 작업 경로 작성 후 실행(구글 드라이브 최상위에 압축해제 시 그대로 실행. 수정 X).<br>

<font color="red">※ 주의. 경로는 절대 변경하지 마세요.</font>

In [ ]:
# ROOT_PATH 확인 
import os

# 구글 드라이브 내 프로젝트 압축해제된 영역 (구글 드라이브 최상위에 압축해제 시 그대로 실행 수정 X)
WORK_SPACE = ""

if os.getcwd() == '/content' :
  # 구글 드라이브 사용 시 
  ROOT_PATH = "/content/drive/MyDrive/"+WORK_SPACE+"/AIVLE3rd_individual"
else :
  ROOT_PATH = os.path.abspath('..')

# 모델 예측을 위한 test 데이터가 저장되는 경로
TEST_PATH = ROOT_PATH + "/test"
# 테스트 영상 test.mp4 경로 

TEST_VIDEO = TEST_PATH + "/test.mp4"

# 모델 예측을 위해 테스트 영상을 프레임 이미지로 자르고 저장하는 경로 (testGenerator 생성 시 PATH )   
TEST_IMAGE = TEST_PATH + "/image"

# 모델(.h5) 파일이 저장된 경로 (본인이 생성한 모델)
MODEL_PATH = ROOT_PATH + "/model"

### 3) 테스트 데이터 다운로드하기
아래 코드를 실행하세요.

In [ ]:
# test data 다운로드
!pip install gdown

import gdown
import shutil
import zipfile
 
google_path = 'https://drive.google.com/uc?id='
file_id = "10iKCHLPx-YFgkJcDqCgJPl83ZKYHvXmr"
output_name = 'test.zip'

# 파일 다운로드
gdown.download(google_path+file_id,output_name,quiet=False)

# 파일 위치 이동
shutil.move("./" + output_name, ROOT_PATH)

zip_file = ROOT_PATH + "/" + output_name

# 압축해제 
with zipfile.ZipFile(zip_file) as z:
    z.extractall(ROOT_PATH)

os.remove(zip_file)


---

## 1. 데이터 전처리 함수 만들기
### 전처리 함수 (my_preprocessing) <br>
_[study] 2. 데이터 전처리.ipynb 파일을 참고하세요._
> - 함수명 : my_preprocessing
> - 설명 : 모델 예측을 위한 테스트 데이터에 적용할 전처리 과정 
> - 매개변수 (parameter)<br>
    ① TEST_VIDEO : (string) test.mp4 파일 경로<br>
    ② TEST_IMAGE : (string) 모델 예측을 위해 테스트 영상을 프레임 이미지로 자르고 저장하는 경로 <br>
     ※ <u>test_generator 생성 시 TEST_PATH 내 1개의 Class(폴더) 반드시 필요</u>
> - 기능<br>
  * [필수]
    * TEST_VIDEO(test.mp4) 영상을 <b>1초</b> 단위로 프레임(<u>이미지 1220장</u>)을 추출하는 기능 <br>
    * 자신이 만든 모델로 TEST_IMAGE 예측을 위한 keras ImageDataGenerator 생성 <br>
> - 반환값 (return value)
>> test_generator : TEST_PATH를 경로로 하여 ImageDataGenerator로 생성한 값  

<font color="red">[주의사항]</font><br>
전처리 함수 실행을 위한 필요 라이브러리가 있을 경우 함수 안에서 호출하세요. 
전처리는 my_preprocssing 함수만 실행할 예정입니다.
라이브러리가 import 되지 않았을 경우 실행이 안될 수 있습니다.  

In [ ]:
def my_preprocessing(TEST_VIDEO):
  # 여기서 부터 작성하세요.
 
 
 








 
  return test_generator

In [ ]:
# 결과 확인 (수정하지 마세요.) - Found 1220 images belonging to 1 classes. 와 같은 메시지가 출력되어야 합니다. 
test_generator = my_preprocessing(TEST_VIDEO)

Found 1220 images belonging to 1 classes.


## 2. 모델 적용 예측결과 출력 함수 만들기
### 모델 적용 예측결과 출력 함수 (my_model_predict) <br>
_[study] 3. AI모델링(딥러닝).ipynb 파일을 참고하세요._
> - 함수명 : my_model_predict
> - 설명 : 자신의 모델로 TEST_IMAGE를 예측한 결과 출력 함수
> - 매개변수 (parameter)<br>
    ① test_generator : (ImageDataGenerator) 전처리 함수를 통해 반환한 test_generator <br>
    ② MODEL_PATH : (string) 자신의 모델이 저장된 경로 <br>
> - 기능<br>
  * [필수]
    * 자신이 만든 모델을 불러오고(load_model), 예측하기(modle.predict) 기능 
    * 예측 결과를 데이터프레임 형태로 ['파일명(또는 시간)', '예측결과']으로 출력  <br>
    ※ <u>예측 결과 출력에는 별도의 양식은 없으며, 결과를 보고 1~1220초 사이에 어떤 출연자(코너)인지만 확인 가능하면 상관 없습니다.</u>
> - 반환값 (return value)
>> predict_df : (DATAFRAME) 예측 결과를 저장한 데이터 프레임 

<font color="red">[주의사항]</font><br>
함수 실행을 위한 필요 라이브러리가 있을 경우 함수 안에서 호출하세요. 
모델 적용 예측결과 출력 함수(my_model_predict)만 실행할 예정입니다. 라이브러리가 import 되지 않았을 경우 실행이 안될 수 있습니다.  



In [ ]:
def my_model_predict(test_generator, MODEL_PATH):
  # 여기서 부터 작성하세요.
 









 



  return predict_df

In [ ]:
# 결과 확인 (수정하지 마세요.)
predict_df = my_model_predict(test_generator, MODEL_PATH)